## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [14:30,  8.62it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:51<00:00, 146.35it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Dr Martin Daly,"Ballygar, Co.Galway",Rural GP Sports Med Doc,2012-08-24 10:48:02,2211,1726,3172,False,2020-08-17 08:24:38,"@Mulliganj People breaking rules everywhere, n...",None,Twitter for iPhone,False
0,Mø -,LifeArtist&ModernGentleman,•JLS• 1_Race:HumanRace•Free&Fair•Fascist_Intol...,2012-05-30 20:57:10,1298,3690,81051,False,2020-08-17 08:24:37,@CDCgov This is major disinformation. Gently r...,None,Twitter for iPhone,False
0,TheZimbabweSentinel,Harare,"Deeper analysis of political, #economic, socia...",2014-07-03 13:00:07,7056,4119,1742,False,2020-08-17 08:24:36,2. #Gimboki South residents committee &amp; de...,[Gimboki],Twitter Web App,False
0,FORTIUS Solutions,"Kolkata, India",Statistical Analysts | Market Analysts | Busin...,2018-12-25 09:22:04,6,7,22,False,2020-08-17 08:24:36,Daily surge rates in Regions with over 1 lakh ...,"[COVID19, India]",Twitter for Android,False
0,Coronavirus Updates,,COVID-19 (Coronavirus) Latest News & Statistic...,2020-03-07 12:52:51,502,1,0,False,2020-08-17 08:24:26,5 new cases in Slovakia \n\n[9:24 GMT] #corona...,"[coronavirus, CoronaVirusUpdate, COVID19, Coro...",COVID19-Updates,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (140101, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 140101 all tweets: 147601


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (147601, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)